# Merging Ordered and Time-Series Data

## Correlation between GDP and S&P500
In this exercise, you want to analyze stock returns from the S&P 500. You believe there may be a relationship between the returns of the S&P 500 and the GDP of the US. Merge the different datasets together to compute the correlation.

* Use `merge_ordered()` to merge gdp and sp500 using a left join on year and date. Save the results as `gdp_sp500`.
* Print `gdp_sp500` and look at the returns for the year 2018.

In [3]:
import pandas as pd

sp500 = pd.read_csv('C:/Users/Александр/pj/DataCamp_projects/Data Scientist With Python/Joining Data with pandas/datasets/S&P500.csv')
gdp = pd.read_csv('C:/Users/Александр/pj/DataCamp_projects/Data Scientist With Python/Joining Data with pandas/datasets/WorldBank_GDP.csv')

In [ ]:
# Use merge_ordered() to merge gdp and sp500 on year and date
gdp_sp500 = pd.merge_ordered(gdp, sp500, left_on='year', right_on='date', 
                             how='left')
# Print gdp_sp500
print (gdp_sp500)

* Use `merge_ordered()`, again similar to before, to merge gdp and sp500 use the function's ability to interpolate missing data to forward fill the missing value for returns, assigning this table to the variable `gdp_sp500`

In [ ]:
# Use merge_ordered() to merge gdp and sp500, interpolate missing value
gdp_sp500 = pd.merge_ordered(gdp, sp500, left_on='year', right_on='date', 
                             how='left',  fill_method='ffill')

# Print gdp_sp500
print (gdp_sp500)

* Subset the `gdp_sp500` table, select the gdp and returns columns, and save as `gdp_returns`.
* Print the correlation matrix of the `gdp_returns` table.

In [ ]:
# Use merge_ordered() to merge gdp and sp500, interpolate missing value
gdp_sp500 = pd.merge_ordered(gdp, sp500, left_on='year', right_on='date', 
                             how='left',  fill_method='ffill')

# Subset the gdp and returns columns
gdp_returns = gdp_sp500[['gdp','returns']]

# Print gdp_returns correlation
print(gdp_returns.corr())

Awesome work! You can see the different aspects of merge_ordered() and how you might use it on data that can be ordered. By using this function, you were able to fill in the missing data from 2019. Finally, the correlation of 0.21 between the GDP and S&P500 is low to moderate at best. You may want to find another predictor if you plan to play in the stock market.

## Phillips curve using merge_ordered()
There is an economic theory developed by A. W. Phillips which states that inflation and unemployment have an inverse relationship. The theory claims that with economic growth comes inflation, which in turn should lead to more jobs and less unemployment.

You will take two tables of data from the U.S. Bureau of Labor Statistics, containing unemployment and inflation data over different periods, and create a Phillips curve. The tables have different frequencies. One table has a data entry every six months, while the other has a data entry every month. You will need to use the entries where you have data within both tables.

* Use `merge_ordered()` to merge the inflation and unemployment tables on date with an inner join, and save the results as `inflation_unemploy`.
* Print the `inflation_unemploy` variable.
*  Using `inflation_unemploy`, create a scatter plot with `unemployment_rate` on the horizontal axis and cpi (inflation) on the vertical axis.

In [ ]:
# Use merge_ordered() to merge inflation, unemployment with inner join
inflation_unemploy = pd.merge_ordered(inflation, unemployment, 
                                      on='date', how='inner')

# Print inflation_unemploy 
print(inflation_unemploy)

# Plot a scatter plot of unemployment_rate vs cpi of inflation_unemploy
inflation_unemploy.plot(kind='scatter', x='unemployment_rate', y='cpi')
plt.show()



Great work! You created a Phillips curve. There are critics of the curve, but what is more important in this example is that you were able to use entries where you had entries in both tables by using an inner join. You might ask why not use the default outer join and use forward fill to fill to estimate the missing variables. You might choose differently. In this case, instead of showing an estimated unemployment rate (which is a continually changing measure) for five periods, that data was dropped from the plot.

## merge_ordered() caution, multiple columns
When using `merge_ordered()` to merge on multiple columns, the order is important when you combine it with the forward fill feature. The function sorts the merge on columns in the order provided. In this exercise, we will merge GDP and population data from the World Bank for the Australia and Sweden, reversing the order of the merge on columns. The frequency of the series are different, the GDP values are quarterly, and the population is yearly. Use the forward fill feature to fill in the missing data. Depending on the order provided, the fill forward will use unintended data to fill in the missing values.

* Use `merge_ordered()` on gdp and pop, merging on columns date and country with the fill feature, save to `ctry_date`.

In [ ]:
pop = pd.read_csv('C:/Users/Александр/pj/DataCamp_projects/Data Scientist With Python/Joining Data with pandas/datasets/WorldBank_POP.csv')

# Merge gdp and pop on date and country with fill and notice rows 2 and 3
ctry_date = pd.merge_ordered(gdp, pop, on=['date','country'], 
                             fill_method='ffill')

# Print ctry_date
print(ctry_date)

* Perform the same merge of gdp and pop, but join on country and date (reverse of step 1) with the fill feature, saving this as `date_ctry`.

In [ ]:
# Merge gdp and pop on country and date with fill
date_ctry = pd.merge_ordered(gdp, pop, on=['country','date'], 
                             fill_method='ffill')

# Print date_ctry
print(date_ctry)

Nice! When you merge on date first, the table is sorted by date then country. When forward fill is applied, Sweden's population value in January is used to fill in the missing values for both Australia and the Sweden for the remainder of the year. This is not what you want. The fill forward is using unintended data to fill in the missing values. However, when you merge on country first, the table is sorted by country then date, so the forward fill is applied appropriately in this situation.

## Using merge_asof() to study stocks
You have a feed of stock market prices that you record. You attempt to track the price every five minutes. Still, due to some network latency, the prices you record are roughly every 5 minutes. You pull your price logs for three banks, JP Morgan (JPM), Wells Fargo (WFC), and Bank Of America (BAC). You want to know how the price change of the two other banks compare to JP Morgan. Therefore, you will need to merge these three logs into one table. Afterward, you will use the pandas `.diff()` method to compute the price change over time. Finally, plot the price changes so you can review your analysis.

* Use `merge_asof()` to merge jpm (left table) and wells together on the `date_time` column, where the rows with the nearest times are matched, and with `suffixes=('', '_wells')`. Save to `jpm_wells`.
* Use `merge_asof()` to merge jpm_wells (left table) and bac together on the `date_time` column, where the rows with the closest times are matched, and with `suffixes=('_jpm', '_bac')`. Save to `pm_wells_bac`.
* Using `price_diffs`, create a line plot of the close price of JPM, WFC, and BAC only.

In [ ]:
# Use merge_asof() to merge jpm and wells
jpm_wells = pd.merge_asof(jpm, wells, on='date_time', 
                          suffixes=('', '_wells'), direction='nearest')

# Use merge_asof() to merge jpm_wells and bac
jpm_wells_bac = pd.merge_asof(jpm_wells, bac, on='date_time', 
                              suffixes=('_jpm', '_bac'), direction='nearest')

# Compute price diff
price_diffs = jpm_wells_bac.diff()

# Plot the price diff of the close of jpm, wells and bac only
price_diffs.plot(y=['close_jpm','close_wells','close_bac'])
plt.show()



Fabulous! You can see that during this period, the price change for these bank stocks was roughly the same, although the price change for JP Morgan was more variable. The critical point here is that the merge_asof() function is very useful in performing the fuzzy matching between the timestamps of all the tables.

## Using merge_asof() to create dataset
The `merge_asof()` function can be used to create datasets where you have a table of start and stop dates, and you want to use them to create a flag in another table. You have been given gdp, which is a table of quarterly GDP values of the US during the 1980s. Additionally, the table recession has been given to you. It holds the starting date of every US recession since 1980, and the date when the recession was declared to be over. Use merge_asof() to merge the tables and create a status flag if a quarter was during a recession. Finally, to check your work, plot the data in a bar chart.

* Using `merge_asof()`, merge gdp and recession on date, with gdp as the left table. Save to the variable `gdp_recession`.
* Create a list using a list comprehension and a conditional expression, named `is_recession`, where for each row if the `gdp_recession['econ_status']` value is equal to 'recession' then enter 'r' else 'g'.
* Using `gdp_recession`, plot a bar chart of gdp versus date, setting the color argument equal to `is_recession`.

In [ ]:
# Merge gdp and recession on date using merge_asof()
gdp_recession = pd.merge_asof(gdp, recession, on='date')

# Create a list based on the row value of gdp_recession['econ_status']
is_recession = ['r' if s=='recession' else 'g' for s in gdp_recession['econ_status']]

# Plot a bar chart of gdp_recession
gdp_recession.plot(kind='bar', y='gdp', x='date', color=is_recession, rot=90)
plt.show()



Terrific work! You can see from the chart that there were a number of quarters early in the 1980s where a recession was an issue. merge_asof() allowed you to quickly add a flag to the gdp dataset by matching between two different dates, in one line of code! If you were to perform the same task using subsetting, it would have taken a lot more code.

## Explore financials with .query()
You have been given a table of financial data from some popular social network companies called `social_fin`. All of the values are in thousands of US dollars.

Use the `.query()` method and the IPython shell to explore social_fin and select the True statement.ent into the appropriate box for either the `merge_asof()` function, the `merge_ordered()` function, or both if it applies to both functions.


* There 2 rows where the value is greater than $50,000,000K.

* There are 3 rows for total revenue for Facebook.

* **There are 6 rows where the net income has a negative value.**

* There are 45 rows, where the gross profit is greater than $100K.

That's correct! To check this answer you needed to select financial equal to net income and value < 0.

## Subsetting rows with .query()
In this exercise, you will revisit GDP and population data for Australia and Sweden from the World Bank and expand on it using the `.query()` method. You'll merge the two tables and compute the GDP per capita. Afterwards, you'll use the `.query()` method to sub-select the rows and create a plot. Recall that you will need to merge on multiple columns in the proper order.

* Use `merge_ordered()` on gdp and pop on columns country and date with the fill feature, save to `gdp_pop` and print.

In [ ]:
# Merge gdp and pop on date and country with fill
gdp_pop = pd.merge_ordered(gdp, pop, on=['country','date'], fill_method='ffill')

* Add a column named `gdp_per_capita` to gdp_pop that divides gdp by pop.

In [ ]:
# Add a column named gdp_per_capita to gdp_pop that divides the gdp by pop
gdp_pop['gdp_per_capita'] = gdp_pop['gdp'] / gdp_pop['pop']

* Pivot `gdp_pop` so values='gdp_per_capita', index='date', and columns='country', save as `gdp_pivot`.

In [ ]:
# Pivot table of gdp_per_capita, where index is date and columns is country
gdp_pivot = gdp_pop.pivot_table('gdp_per_capita', 'date', 'country')

* Use `.query()` to select rows from `gdp_pivot` where date is greater than equal to "1991-01-01". Save as `recent_gdp_pop`.

In [ ]:
# Select dates equal to or greater than 1991-01-01
recent_gdp_pop = gdp_pivot.query('date >= "1991-01-01"')

# Plot recent_gdp_pop
recent_gdp_pop.plot(rot=90)
plt.show()



Amazing! You can see from the plot that the per capita GDP of Australia passed Sweden in 1992. By using the .query() method, you were able to select the appropriate rows easily. The .query() method is easy to read and straightforward.

## Select the right .melt() arguments
You are given a table named inflation. Chose the option to get the same output as the table below.
```
   country    indicator  year  annual
0   Brazil  Inflation %  2017    3.45
1   Canada  Inflation %  2017    1.60
2   France  Inflation %  2017    1.03
3    India  Inflation %  2017    2.49
4   Brazil  Inflation %  2018    3.66
5   Canada  Inflation %  2018    2.27
6   France  Inflation %  2018    1.85
7    India  Inflation %  2018    4.86
8   Brazil  Inflation %  2019    3.73
9   Canada  Inflation %  2019    1.95
10  France  Inflation %  2019    1.11
11   India  Inflation %  2019    7.66
```

* inflation.melt(id_vars=['country','indicator'], var_name='annual')

* inflation.melt(id_vars=['country'], var_name='indicator', value_name='annual')

* **inflation.melt(id_vars=['country','indicator'], var_name='year', value_name='annual')**

* inflation.melt(id_vars=['country'], var_name='year', value_name='annual')

Magnificent! You identified the correct values to pass to the id_vars argument. These columns are not unpivoted. Finally, the other arguments set the name for the year and value columns.

## Using .melt() to reshape government data
The US Bureau of Labor Statistics (BLS) often provides data series in an easy-to-read format - it has a separate column for each month, and each year is a different row. Unfortunately, this wide format makes it difficult to plot this information over time. In this exercise, you will reshape a table of US unemployment rate data from the BLS into a form you can plot using `.melt()`. You will need to add a date column to the table and sort by it to plot the data correctly.

The unemployment rate data has been loaded for you in a table called `ur_wide`. You are encouraged to view the table in the IPython shell before beginning the exercise.

* Use `.melt()` to unpivot all of the columns of `ur_wide` except year and ensure that the columns with the months and values are named month and `unempl_rate`, respectively. Save the result as `ur_tall`.
* Add a column to `ur_tall` named date which combines the year and month columns as year-month format into a larger string, and converts it to a date data type.
* Sort `ur_tall` by date and save as ur_sorted.
* Using `ur_sorted`, plot unempl_rate on the y-axis and date on the x-axis.

In [ ]:
# Unpivot everything besides the year column
ur_tall = ur_wide.melt(id_vars=['year'], var_name='month', 
                       value_name='unempl_rate')

# Create a date column using the month and year columns of ur_tall
ur_tall['date'] = pd.to_datetime(ur_tall['month'] + '-' + ur_tall['year'])

# Sort ur_tall by date in ascending order
ur_sorted = ur_tall.sort_values('date')

# Plot the unempl_rate by date
ur_sorted.plot(x='date', y='unempl_rate')
plt.show()



Nice going! The plot shows a steady decrease in the unemployment rate with an increase near the end. This increase is likely the effect of the COVID-19 pandemic and its impact on shutting down most of the US economy. In general, data is often provided (especially by governments) in a format that is easily read by people but not by machines. The .melt() method is a handy tool for reshaping data into a useful form.

## Using .melt() for stocks vs bond performance
It is widespread knowledge that the price of bonds is inversely related to the price of stocks. In this last exercise, you'll review many of the topics in this chapter to confirm this. You have been given a table of percent change of the US 10-year treasury bond price. It is in a wide format where there is a separate column for each year. You will need to use the .melt() method to reshape this table.

Additionally, you will use the `.query()` method to filter out unneeded data. You will merge this table with a table of the percent change of the Dow Jones Industrial stock index price. Finally, you will plot data.

* Use `.melt()` on `ten_yr` to unpivot everything except the metric column, setting var_name='date' and value_name='close'. Save the result to `bond_perc`.
* Using the `.query()` method, select only those rows were metric equals 'close', and save to `bond_perc_close`.
* Use `merge_ordered()` to merge dji (left table) and `bond_perc_close` on date with an inner join, and set suffixes equal to ('_dow', '_bond'). Save the result to `dow_bond`.
* Using `dow_bond`, plot only the Dow and bond values.

In [ ]:
# Use melt on ten_yr, unpivot everything besides the metric column
bond_perc = ten_yr.melt(id_vars='metric', var_name='date', value_name='close')

# Use query on bond_perc to select only the rows where metric=close
bond_perc_close = bond_perc.query('metric == "close"')

# Merge (ordered) dji and bond_perc_close on date with an inner join
dow_bond = pd.merge_ordered(dji, bond_perc_close, on='date', 
                            suffixes=('_dow', '_bond'), how='inner')

# Plot only the close_dow and close_bond columns
dow_bond.plot(y=['close_dow', 'close_bond'], x='date', rot=90)
plt.show()



Super job! You used many of the techniques we have reviewed in this chapter to produce the plot. The plot confirms that the bond and stock prices are inversely correlated. Often as the price of stocks increases, the price for bonds decreases.